In [20]:
def formatter(game):
    
    game = game.translate(str.maketrans({':': '', ' ': '-'}))

    return game.lower()
# Combine all the data, both games and game details
import aiohttp
import asyncio
import pandas as pd
import requests
import csv

async def retrieve_games_info(game_name):
    # API endpoint to retrieve game details
    endpoint = "https://www.giantbomb.com/api/search"

    # API key, you'll need to sign up for a free API key to use the GiantBomb API
    api_key = "69fe0db0bb287eeb835b3edcbab1388545aeb099"

    # Query parameters for the API request
    params = {
        "api_key": api_key,
        "format": "json",
        "query": game_name,
        "resources": "game",
        "field_list": "name,expected_release_day,platforms,characters,dlcs,concepts,franchises,genres,regions,releases,description",
        "limit": 1000 # Limit the number of results to 1000
    }

    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
    "Accept": "*/*",
    "Referer": "https://www.giantbomb.com",
    "Connection": "keep-alive"
    }

    async with aiohttp.ClientSession(connector=aiohttp.TCPConnector(ssl=False)) as session:
        async with session.get(endpoint, params=params, headers=headers) as response:
            # Check if the response was successful
            if response.status == 200:
                # Extract the games information from the response
                response_json = await response.json()
                #print(game_name)
                games_info = response_json["results"]
                # Filter the games to include only the ones available on Steam
                #steam_games = [game for game in games_info if "Steam" in [platform["name"] for platform in game["platforms"]]]
                #print(steam_games)
                return games_info
            else:
                raise Exception(f"Failed to retrieve games info. Status code: {response.status}")

async def fetch_game_details(s,appid):
    async with aiohttp.ClientSession(connector=aiohttp.TCPConnector(ssl=False)) as session:
        async with session.get(f"https://steamspy.com/api.php?request=appdetails&appid={appid}") as resp:
            return await resp.json()

async def fetch_games(session):
    async with session.get("https://api.steampowered.com/ISteamApps/GetAppList/v2") as resp:
        return await resp.json()

async def save_games_to_csv(session,games):
    games_data = []
    c = 0
    for game in games["applist"]["apps"]:
        print(c)
        if c == 100:
            break
        c+= 1
        game_details = await fetch_game_details(session, game["appid"])
        try:
            further_details = await retrieve_games_info(formatter(game['name']))
            print(further_details[0])
        except:
            continue

        if game_details:
            data = game_details
            #games_data.append([game["appid"], game["name"], data.get("price", "0"), data.get("developer", ""), data.get("publisher", ""), data.get("average_forever", 0), data.get("median_forever", 0), data.get("ccu", 0), data.get("genre", []), data.get("languages", []), data.get("score_rank", 0)])
            if further_details:
                #further_data = further_details
                #games_data[-1].extend([g.get("description", 0), g.get("expected_release_day", 0), g.get("original_release_date", 0), ", ".join([character["name"] for character in g.get("characters", [])]), ", ".join([dlc["name"] for dlc in g.get("dlcs", [])]), ", ".join([concept["name"] for concept in g.get("concepts", [])]), ", ".join([franchise["name"] for franchise in g.get("franchises", [])]), ", ".join([genre["name"] for genre in g.get("genres", [])]), ", ".join([region["name"] for region in g.get("regions", [])]), ", ".join([release["name"] for release in g.get("releases", [])])])
                games_data.append([
                    game["appid"], game["name"], 
                    data.get("price", "0"), data.get("developer", ""), data.get("publisher", ""), 
                    data.get("average_forever", 0), data.get("median_forever", 0), data.get("ccu", 0), 
                    data.get("genre", []), data.get("languages", []), data.get("score_rank", 0), 
                    further_details[0].get("description", ""), 
                    further_details[0].get("expected_release_day", ""), 
                    further_details[0].get("original_release_date", ""), 
                    ", ".join([character["name"] for character in further_details[0].get("characters", [])]), 
                    ", ".join([dlc["name"] for dlc in further_details[0].get("dlcs", [])]), 
                    ", ".join([concept["name"] for concept in further_details[0].get("concepts", [])]), 
                    ", ".join([franchise["name"] for franchise in further_details[0].get("franchises", [])]), 
                    ", ".join([genre["name"] for genre in further_details[0].get("genres", [])]), 
                    ", ".join([region["name"] for region in further_details[0].get("regions", [])]), 
                    ", ".join([release["name"] for release in further_details[0].get("releases", [])])
                ])
        else:
            continue

    df = pd.DataFrame(games_data, columns=["appid", "name", "price", "developer", "publisher", "average_forever", "median_forever", "ccu", "genres", "languages" ,"score_rank", "description", "expected_release_day", "original_release_date", "characters", "dlcs", "concepts", "franchises", "genres", "regions", "releases"])
    df.to_csv("all_games.csv", index=False)
    return df

async def main():
    async with aiohttp.ClientSession() as session:
        games = await fetch_games(session)
        await save_games_to_csv(session, games)

if __name__ == "__main__":
    await main()

0
1
2
3
4
5
6
7
{'description': None, 'expected_release_day': None, 'name': "Darwin's Test", 'platforms': [{'api_detail_url': 'https://www.giantbomb.com/api/platform/3045-94/', 'id': 94, 'name': 'PC', 'site_detail_url': 'https://www.giantbomb.com/pc/3045-94/', 'abbreviation': 'PC'}], 'resource_type': 'game'}
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
{'description': None, 'expected_release_day': None, 'name': 'Réussir : Code la Route', 'platforms': [{'api_detail_url': 'https://www.giantbomb.com/api/platform/3045-17/', 'id': 17, 'name': 'Mac', 'site_detail_url': 'https://www.giantbomb.com/mac/3045-17/', 'abbreviation': 'MAC'}, {'api_detail_url': 'https://www.giantbomb.com/api/platform/3045-94/', 'id': 94, 'name': 'PC', 'site_detail_url': 'https://www.giantbomb.com/pc/3045-94/', 'abbreviation': 'PC'}, {'api_detail_url': 'https://www.giantbomb.com/api/platform/3045-146/', 'id': 146, 'name': 'PlayStation 4', 'site_detail_url': 'https://www.giantbomb.com/playst